# Mamba-GNN Training Notebook (EstraNet-Aligned Configuration)

**⚠️ IMPORTANT**: This notebook uses the **corrected configuration** that matches EstraNet for fair comparison.

## Key Configuration ✅
- **Loss Function**: Cross-Entropy (not Focal Loss)
- **Learning Rate**: 2.5e-4 (not 2e-3)
- **Batch Size**: 256 train / 32 eval (not 64)
- **Training**: 100k steps (not 50-100 epochs)
- **Optimizer**: Adam (not AdamW)
- **Scheduler**: Cosine Decay (not OneCycleLR)
- **Evaluation**: 100-trial Guessing Entropy (not single trial)

## DO NOT USE `final_best_gnn_mamba_teacher.ipynb` ❌
That notebook has incorrect configuration and cannot be fairly compared with EstraNet!

In [ ]:
import os, shutil

os.chdir('/content')
if os.path.exists('EstraNet'):
    shutil.rmtree('EstraNet')  # Remove nested mess

!git clone https://github.com/loshithan/EstraNet.git
os.chdir('EstraNet')
print(f"✅ Clean! Directory: {os.getcwd()}")

Cloning into 'EstraNet'...
remote: Enumerating objects: 310, done.
remote: Counting objects: 100% (310/310), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 310 (delta 132), reused 271 (delta 93), pack-reused 0 (from 0)
Receiving objects: 100% (310/310), 32.22 MiB | 16.21 MiB/s, done.
Resolving deltas: 100% (132/132), done.
✅ Clean! Directory: /content/EstraNet


## Setup Environment

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Add paths
sys.path.append(str(Path.cwd() / 'mamba-gnn-scripts'))
sys.path.append(str(Path.cwd()))

# Configuration
DATA_PATH = 'data/ASCAD.h5'
CHECKPOINT_DIR = 'checkpoints/mamba_gnn_estranet'
RESULT_DIR = 'results'

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

print(f"\n✓ Environment ready")
print(f"  Data path: {DATA_PATH}")
print(f"  Checkpoint dir: {CHECKPOINT_DIR}")
print(f"  Result dir: {RESULT_DIR}")

PyTorch version: 2.9.0+cu128
CUDA available: True
CUDA device: NVIDIA A100-SXM4-40GB

✓ Environment ready
  Data path: data/ASCAD.h5
  Checkpoint dir: checkpoints/mamba_gnn_estranet
  Result dir: results


## Configuration Comparison

### ❌ Old Notebook (final_best_gnn_mamba_teacher.ipynb)
```python
# WRONG CONFIGURATION - DO NOT USE
loss = FocalLoss(gamma=2.5)
optimizer = AdamW(lr=2e-3)
batch_size = 64
epochs = 50-100
scheduler = OneCycleLR(max_lr=2e-3)
evaluation = single_trial_key_rank()
```

### ✅ New Configuration (This Notebook)
```python
# CORRECT CONFIGURATION - Matches EstraNet
loss = CrossEntropyLoss()
optimizer = Adam(lr=2.5e-4)
batch_size = 256 (train), 32 (eval)
train_steps = 100000
scheduler = CosineLRSchedule(max_lr=2.5e-4)
evaluation = compute_ge_key_rank(num_trials=100)
```

## Option 1: Train Using PyTorch Script (Recommended)

This uses the corrected `train_mamba_gnn.py` script with EstraNet-aligned configuration.

## Choose Loss Function: Fair Comparison vs Performance

**Two-Phase Experiment Strategy for Your Report:**

### Phase 1️⃣ (FIRST): Fair Comparison with Cross-Entropy
- **Purpose**: Compare Mamba-GNN vs EstraNet objectively (same configuration)
- **Loss**: Cross-Entropy
- **Results**: Document baseline performance
- **Section in report**: "Fair Comparison Analysis"

### Phase 2️⃣ (THEN): Best Performance with FocalLoss
- **Purpose**: Show Mamba-GNN's maximum attack potential
- **Loss**: FocalLoss (gamma=2.5)
- **Results**: Document optimized performance
- **Section in report**: "Performance Analysis" or "Best-Case Scenario"

Toggle loss function below ⬇️

In [ ]:
# ========== SWITCH LOSS FUNCTION HERE ==========
# Choose one: "cross_entropy" or "focal_loss"
LOSS_FUNCTION = "cross_entropy"  # ← Change to "focal_loss" for Phase 2
# ===============================================

# Training configuration (EstraNet-aligned + Anti-Overfitting Regularization)
config = {
    'data_path': DATA_PATH,
    'checkpoint_dir': CHECKPOINT_DIR,
    'target_byte': 2,
    'train_batch_size': 256,
    'eval_batch_size': 32,
    'train_steps': 25000,        # ✅ Reduced from 100000 to 25000
    'learning_rate': 2.5e-4,
    'd_model': 64,              # ⚠️ REDUCED from 128 to prevent overfitting
    'mamba_layers': 2,          # ⚠️ REDUCED from 4 to prevent overfitting
    'gnn_layers': 2,            # ⚠️ REDUCED from 3 to prevent overfitting
    'k_neighbors': 8,
    'dropout': 0.3,             # ⚠️ INCREASED from 0.1 to prevent overfitting
    'iterations': 500,
    'eval_steps': 250,          # ✅ More frequent evaluation to detect overfitting early
    'save_steps': 5000,         # ✅ Checkpoints every 5k steps
    'clip': 0.25,
    'warmup_steps': 1000,
    'loss_function': LOSS_FUNCTION,
    
    # NEW: Anti-overfitting regularization
    'weight_decay': 0.01,       # L2 regularization
    'label_smoothing': 0.1,     # Soften targets
    'early_stopping': 5,        # ⚠️ Reduced patience to react quicker (eval periods)
    'augment_noise': 0.05,      # Gaussian noise std
    'augment_shift': 3,         # Max random time shift
}

# Loss function specific settings
if LOSS_FUNCTION == "cross_entropy":
    config['loss_config'] = {
        'type': 'CrossEntropyLoss',
        'description': 'Fair comparison with EstraNet',
    }
    print("\n" + "="*80)
    print("PHASE 1️⃣: FAIR COMPARISON MODE (WITH REGULARIZATION)")
    print("="*80)
    print("\n✓ Loss Function: Cross-Entropy (with label smoothing 0.1)")
    print("✓ Regularization: weight_decay=0.01, dropout=0.3")
    print("✓ Data Augmentation: noise=0.05, shift=3")
    print("✓ Model Capacity: REDUCED (d_model=64, mamba=2, gnn=2)")
    print("✓ Early Stopping: After 5 eval periods without improvement")
    print("\n⚠️ Previous training showed SEVERE OVERFITTING:")
    print("   Train loss → 0, Eval loss → 22 (massive gap)")
    print("   These changes should prevent that.")

print("\n✓ Configuration:")
for key, value in config.items():
    if key != 'loss_config':
        print(f"  {key:20s}: {value}")


PHASE 1️⃣: FAIR COMPARISON MODE

✓ Loss Function: Cross-Entropy
✓ Purpose: Compare Mamba-GNN vs EstraNet with same configuration
✓ Results: Document in 'Fair Comparison Analysis' section

✓ Configuration:
  data_path           : data/ASCAD.h5
  checkpoint_dir      : checkpoints/mamba_gnn_estranet
  target_byte         : 2
  train_batch_size    : 256
  eval_batch_size     : 32
  train_steps         : 100000
  learning_rate       : 0.00025
  d_model             : 128
  mamba_layers        : 4
  gnn_layers          : 3
  k_neighbors         : 8
  dropout             : 0.1
  iterations          : 500
  eval_steps          : 500
  save_steps          : 10000
  clip                : 0.25
  warmup_steps        : 1000
  loss_function       : cross_entropy


## Report Structure: How to Document Results

### Section 1: Fair Comparison Analysis (Phase 1️⃣)
```markdown
## Fair Comparison: Mamba-GNN vs EstraNet

### Configuration Match
- Loss: Cross-Entropy (same as EstraNet) ✓
- Learning Rate: 2.5e-4 (same as EstraNet) ✓
- Batch Size: 256 (same as EstraNet) ✓
- Optimizer: Adam with Cosine schedule ✓
- Evaluation: 100-trial Guessing Entropy ✓

### Results
| Model | Key Rank @ 100 traces | Key Rank @ 1000 traces | Recovery Point |
|-------|----------------------|----------------------|--------|
| EstraNet Transformer | X | Y | Z traces |
| Mamba-GNN | X | Y | Z traces |

### Conclusion
Fair comparison shows [which model is better/comparable]
```

### Section 2: Performance Analysis (Phase 2️⃣)
```markdown
## Performance Optimization: Mamba-GNN with FocalLoss

### Configuration
- Loss: FocalLoss (gamma=2.5) - optimized for attack
- Same training setup otherwise ✓

### Results
| Model | Loss | Key Rank @ 100 traces | Key Rank @ 1000 traces |
|-------|------|----------------------|----------------------|
| Mamba-GNN (Cross-Entropy) | Baseline | X | Y |
| Mamba-GNN (FocalLoss) | Optimized | X' | Y' |

### Improvement
FocalLoss improves performance by [X%] at key recovery
```

👇 Execute cells below to gather data for both sections

In [ ]:
import os
import subprocess
from pathlib import Path

# Use absolute script path to ensure the edited script is executed
script_path = os.path.abspath('mamba-gnn-scripts/train_mamba_gnn.py')

# Base args (include all regularization flags we expect)
base_args = {
    'data_path': config['data_path'],
    'checkpoint_dir': config['checkpoint_dir'],
    'target_byte': config['target_byte'],
    'train_batch_size': config['train_batch_size'],
    'eval_batch_size': config['eval_batch_size'],
    'train_steps': config['train_steps'],
    'eval_steps': config['eval_steps'],
    'save_steps': config['save_steps'],
    'learning_rate': config['learning_rate'],
    'd_model': config['d_model'],
    'mamba_layers': config['mamba_layers'],
    'gnn_layers': config['gnn_layers'],
    'k_neighbors': config['k_neighbors'],
    'dropout': config['dropout'],
    'weight_decay': config.get('weight_decay'),
    'label_smoothing': config.get('label_smoothing'),
    'early_stopping': config.get('early_stopping'),
    'augment_noise': config.get('augment_noise'),
    'augment_shift': config.get('augment_shift')
}

# Generate CLI string helper
def build_cli(script, args, allowed_flags=None):
    parts = [f'python "{script}"']
    for k, v in args.items():
        flag = f"--{k}"
        if allowed_flags is not None and flag not in allowed_flags:
            continue
        if v is None:
            continue
        parts.append(f"{flag}={v}")
    parts.append('--do_train')
    return ' \\\n    '.join(parts)

# Probe script --help to see which flags are supported by the runtime script
help_proc = subprocess.run(['python', script_path, '--help'], capture_output=True, text=True)
help_text = (help_proc.stdout or '') + (help_proc.stderr or '')

# Determine allowed flags from help text
expected_flags = [f"--{k}" for k in base_args.keys()]
allowed_flags = set()
for flag in expected_flags:
    if flag in help_text:
        allowed_flags.add(flag)

# If some expected flags are missing, warn and build a cleaned command
missing = [f for f in expected_flags if f not in allowed_flags]
if missing:
    print("⚠️ The training script's --help is missing these flags:", missing)
    print("   Building command without unsupported flags so training can run.")

train_cmd = build_cli(script_path, base_args, allowed_flags)

print(f"Training command ({config['loss_function']}):")
print(train_cmd)
print('\n✓ Run the cell below to start training')

Training command (cross_entropy):
python mamba-gnn-scripts/train_mamba_gnn.py     --data_path=data/ASCAD.h5     --checkpoint_dir=checkpoints/mamba_gnn_estranet     --target_byte=2     --train_batch_size=256     --eval_batch_size=32     --train_steps=100000     --learning_rate=0.00025     --d_model=128     --mamba_layers=4     --gnn_layers=3     --k_neighbors=8     --dropout=0.1     --do_train


✓ Run the cell below to start training


In [ ]:
# Setup checkpoint directories for both phases
import os
from pathlib import Path

# Create phase-specific directories
# FIX: Use the existing directory for Phase 1 as requested
PHASE1_CKPT = 'checkpoints/mamba_gnn_estranet'  # Cross-Entropy
PHASE2_CKPT = 'checkpoints/mamba_gnn_phase2_focal'  # FocalLoss

os.makedirs(PHASE1_CKPT, exist_ok=True)
os.makedirs(PHASE2_CKPT, exist_ok=True)

# Determine which checkpoint dir to use
if config['loss_function'] == "cross_entropy":
    ACTIVE_CKPT = PHASE1_CKPT
    PHASE_NAME = "Phase 1️⃣ (Fair Comparison)"
else:
    ACTIVE_CKPT = PHASE2_CKPT
    PHASE_NAME = "Phase 2️⃣ (Performance)"

config['checkpoint_dir'] = ACTIVE_CKPT

print(f"\n{'='*80}")
print(f"Running: {PHASE_NAME}")
print(f"{'='*80}")
print(f"\n✓ Checkpoints saved to: {ACTIVE_CKPT}")
print(f"✓ Results saved to: {RESULT_DIR}/{config['loss_function']}_eval.txt")
print(f"\nSwitch between phases by changing: LOSS_FUNCTION = '...'")
print(f"{'='*80}\n")


Running: Phase 1️⃣ (Fair Comparison)

✓ Checkpoints saved to: checkpoints/mamba_gnn_estranet
✓ Results saved to: results/cross_entropy_eval.txt

Switch between phases by changing: LOSS_FUNCTION = '...'



In [ ]:
# Execute training with real-time logging and progress monitoring
import subprocess
import sys
import os
from pathlib import Path
import time
from datetime import datetime

# Create log file with timestamp
log_dir = Path("logs")
log_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = log_dir / f"mamba_gnn_training_{timestamp}.log"

print("="*80)
print("Starting Mamba-GNN Training (EstraNet-Aligned Configuration)")
print("="*80)
print(f"\n✓ Command: {train_cmd}")
print(f"✓ Log file: {log_file}\n")

# Modify command to use unbuffered Python output
unbuffered_cmd = train_cmd.replace("python ", "python -u ")

# Monitor checkpoint directory for training progress
checkpoint_dir = Path(config['checkpoint_dir'])
last_checkpoint_time = time.time()

print("Monitoring training progress...\n")

# Open log file for writing
with open(log_file, 'w') as f:
    # Write header
    f.write("="*80 + "\n")
    f.write(f"Mamba-GNN Training Log - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*80 + "\n\n")
    f.write(f"Command: {train_cmd}\n\n")
    f.write("Configuration:\n")
    for key, value in config.items():
        if key != 'loss_config':
            f.write(f"  {key}: {value}\n")
    f.write("\n" + "="*80 + "\n")
    f.write("Training Output:\n")
    f.write("="*80 + "\n\n")
    f.flush()
    
    try:
        # Start training process with unbuffered output
        process = subprocess.Popen(
            unbuffered_cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,  # Line buffering
        )

        # Display output in real-time and log to file
        while True:
            output = process.stdout.readline()
            if output:
                line = output.rstrip()
                print(line)
                f.write(line + "\n")
                f.flush()  # Ensure immediate write

            # Check if process finished
            if process.poll() is not None:
                break

            # Monitor checkpoint creation
            if checkpoint_dir.exists():
                checkpoint_files = list(checkpoint_dir.glob("*.pth"))
                if checkpoint_files:
                    latest = max(checkpoint_files, key=os.path.getctime)
                    current_time = os.path.getctime(latest)
                    if current_time > last_checkpoint_time:
                        msg = f"\n✓ Checkpoint saved: {latest.name}"
                        print(msg)
                        f.write(msg + "\n")
                        f.flush()
                        last_checkpoint_time = current_time

        # Write final status
        if process.returncode == 0:
            msg = "\n" + "="*80 + "\n✓ Training completed successfully!\n" + "="*80
            print(msg)
            f.write(msg + "\n")
        else:
            msg = "\n" + "="*80 + f"\n✗ Training failed with error code: {process.returncode}\n" + "="*80
            print(msg)
            f.write(msg + "\n")

    except Exception as e:
        msg = f"✗ Error running training: {e}"
        print(msg)
        f.write(msg + "\n")

print(f"\n✓ Training log saved to: {log_file}")

Starting Mamba-GNN Training (EstraNet-Aligned Configuration)

✓ Command: python mamba-gnn-scripts/train_mamba_gnn.py     --data_path=data/ASCAD.h5     --checkpoint_dir=checkpoints/mamba_gnn_estranet     --target_byte=2     --train_batch_size=256     --eval_batch_size=32     --train_steps=100000     --learning_rate=0.00025     --d_model=128     --mamba_layers=4     --gnn_layers=3     --k_neighbors=8     --dropout=0.1     --do_train


Monitoring training progress...


CONFIGURATION (EstraNet-aligned)
data_path:         data/ASCAD.h5
target_byte:       2
input_length:      700
train_batch_size:  256
eval_batch_size:   32
train_steps:       100000
learning_rate:     0.00025
d_model:           128
mamba_layers:      4
gnn_layers:        3
checkpoint_dir:    checkpoints/mamba_gnn_estranet

MAMBA-GNN TRAINING (EstraNet-aligned configuration)

Device: cuda
Loading ASCAD from: data/ASCAD.h5
Training traces:  (50000, 700)
Attack traces:    (10000, 700)
Target byte:      2
Optimized Mamba-GNN:
  

KeyboardInterrupt: 

## Monitor Training Progress in Real-Time

Run this cell **in a separate terminal** while training is running to watch checkpoints being saved:

In [ ]:
import os
import time
from pathlib import Path
from datetime import datetime

checkpoint_dir = Path(config['checkpoint_dir'])
previous_files = set()

print("="*80)
print("Training Progress Monitor")
print("="*80)
print(f"\nWatching: {checkpoint_dir}")
print("Press Ctrl+C to stop monitoring\n")

try:
    while True:
        if checkpoint_dir.exists():
            # Get all checkpoint files
            current_files = set(checkpoint_dir.glob("*.pth"))
            new_files = current_files - previous_files

            # Show new checkpoints
            for ckpt_file in sorted(new_files):
                file_size_mb = ckpt_file.stat().st_size / (1024 * 1024)
                timestamp = datetime.fromtimestamp(ckpt_file.stat().st_mtime).strftime("%H:%M:%S")
                print(f"✓ [{timestamp}] {ckpt_file.name} ({file_size_mb:.1f} MB)")

            previous_files = current_files

            # Check total size
            if current_files:
                total_size = sum(f.stat().st_size for f in current_files) / (1024 * 1024)
                print(f"\n✓ Total checkpoints: {len(current_files)}")
                print(f"✓ Total size: {total_size:.1f} MB")
            else:
                print("⏳ Waiting for first checkpoint...")
        else:
            print("⏳ Checkpoint directory not created yet...")

        # Wait before checking again
        time.sleep(5)

except KeyboardInterrupt:
    print("\n\n✓ Monitoring stopped")

Training Progress Monitor

Watching: checkpoints/mamba_gnn_estranet
Press Ctrl+C to stop monitoring

✓ [12:42:29] checkpoint_latest.pth (16.7 MB)
✓ [10:29:46] mamba_gnn-10000.pth (16.7 MB)
✓ [10:56:19] mamba_gnn-20000.pth (16.7 MB)
✓ [11:22:46] mamba_gnn-30000.pth (16.7 MB)
✓ [11:49:11] mamba_gnn-40000.pth (16.7 MB)
✓ [12:15:47] mamba_gnn-50000.pth (16.7 MB)
✓ [12:42:29] mamba_gnn-60000.pth (16.7 MB)

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total size: 117.2 MB

✓ Total checkpoints: 7
✓ Total 

In [ ]:
import os
import time
from pathlib import Path
from datetime import datetime

checkpoint_dir = Path(config['checkpoint_dir'])
previous_files = set()

print("="*80)
print("Training Progress Monitor")
print("="*80)
print(f"\nWatching: {checkpoint_dir}")
print("Press Ctrl+C to stop monitoring\n")

try:
    while True:
        if checkpoint_dir.exists():
            # Get all checkpoint files
            current_files = set(checkpoint_dir.glob("*.pth"))
            new_files = current_files - previous_files

            # Show new checkpoints
            for ckpt_file in sorted(new_files):
                file_size_mb = ckpt_file.stat().st_size / (1024 * 1024)
                timestamp = datetime.fromtimestamp(ckpt_file.stat().st_mtime).strftime("%H:%M:%S")
                print(f"✓ [{timestamp}] {ckpt_file.name} ({file_size_mb:.1f} MB)")

            previous_files = current_files

            # Check total size
            total_size = sum(f.stat().st_size for f in current_files) / (1024 * 1024)
            print(f"\nTotal checkpoints size: {total_size:.1f} MB")
            print(f"Number of checkpoints: {len(current_files)}")
        else:
            print("⚠ Checkpoint directory not found yet...")

        # Wait before checking again
        time.sleep(5)
        print("-" * 80)

except KeyboardInterrupt:
    print("\n✓ Monitoring stopped")


Training Progress Monitor

Watching: checkpoints/mamba_gnn_estranet
Press Ctrl+C to stop monitoring

✓ [12:42:29] checkpoint_latest.pth (16.7 MB)
✓ [10:29:46] mamba_gnn-10000.pth (16.7 MB)
✓ [10:56:19] mamba_gnn-20000.pth (16.7 MB)
✓ [11:22:46] mamba_gnn-30000.pth (16.7 MB)
✓ [11:49:11] mamba_gnn-40000.pth (16.7 MB)
✓ [12:15:47] mamba_gnn-50000.pth (16.7 MB)
✓ [12:42:29] mamba_gnn-60000.pth (16.7 MB)

Total checkpoints size: 117.2 MB
Number of checkpoints: 7
--------------------------------------------------------------------------------

Total checkpoints size: 117.2 MB
Number of checkpoints: 7
--------------------------------------------------------------------------------

Total checkpoints size: 117.2 MB
Number of checkpoints: 7
--------------------------------------------------------------------------------

Total checkpoints size: 117.2 MB
Number of checkpoints: 7
--------------------------------------------------------------------------------

Total checkpoints size: 117.2 MB
Nu

## Option 2: Train Using TensorFlow (for TFLite Conversion)

If you need to deploy on mobile/edge devices, use the TensorFlow version.

In [ ]:
# # TensorFlow training command
# tf_train_cmd = f"""python scripts/train_mamba_gnn_tf.py \
#     --data_path={config['data_path']} \
#     --checkpoint_dir=checkpoints/mamba_gnn_tf \
#     --target_byte={config['target_byte']} \
#     --train_batch_size={config['train_batch_size']} \
#     --eval_batch_size={config['eval_batch_size']} \
#     --train_steps={config['train_steps']} \
#     --learning_rate={config['learning_rate']} \
#     --d_model={config['d_model']} \
#     --do_train
# """

# print("TensorFlow training command:")
# print(tf_train_cmd)
# print("\n✓ This will automatically export to TFLite after training")
# print("  Output: checkpoints/mamba_gnn_tf/mamba_gnn.tflite")

In [ ]:
# Execute TensorFlow training (uncomment to run)
# import subprocess
# result = subprocess.run(tf_train_cmd.split(), capture_output=False)
# print(f"✓ Training complete. TFLite model saved.")

## Monitor Training Progress

View training loss, learning rate, and gradient norms.

In [ ]:
import pickle
from pathlib import Path

# Load training history
loss_path = Path(CHECKPOINT_DIR) / 'loss.pkl'

if loss_path.exists():
    with open(loss_path, 'rb') as f:
        history = pickle.load(f)

    steps = sorted(history.keys())
    train_losses = [history[s]['train_loss'] for s in steps if 'train_loss' in history[s]]
    lrs = [history[s]['lr'] for s in steps if 'lr' in history[s]]
    grad_norms = [history[s]['grad_norm'] for s in steps if 'grad_norm' in history[s]]

    # Plot training metrics
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Training loss
    axes[0].plot(steps[:len(train_losses)], train_losses, 'b-', linewidth=2)
    axes[0].set_xlabel('Training Steps', fontsize=12)
    axes[0].set_ylabel('Loss (Cross-Entropy)', fontsize=12)
    axes[0].set_title('Training Loss', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)

    # Learning rate
    axes[1].plot(steps[:len(lrs)], lrs, 'g-', linewidth=2)
    axes[1].set_xlabel('Training Steps', fontsize=12)
    axes[1].set_ylabel('Learning Rate', fontsize=12)
    axes[1].set_title('Learning Rate Schedule (Cosine Decay)', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)

    # Gradient norm
    axes[2].plot(steps[:len(grad_norms)], grad_norms, 'r-', linewidth=2)
    axes[2].set_xlabel('Training Steps', fontsize=12)
    axes[2].set_ylabel('Gradient Norm', fontsize=12)
    axes[2].set_title('Gradient Norm (Clipped at 0.25)', fontsize=14, fontweight='bold')
    axes[2].axhline(y=0.25, color='orange', linestyle='--', label='Clip threshold')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{RESULT_DIR}/training_progress.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\n✓ Training Progress:")
    print(f"  Total steps: {max(steps):,}")
    print(f"  Latest loss: {train_losses[-1]:.4f}")
    print(f"  Latest LR: {lrs[-1]:.6f}")
    print(f"  Latest grad norm: {grad_norms[-1]:.4f}")
    print(f"\n  Plot saved: {RESULT_DIR}/training_progress.png")
else:
    print("⚠ No training history found. Start training first.")

⚠ No training history found. Start training first.


## Evaluate Model (Guessing Entropy)

Evaluate using 100-trial Guessing Entropy methodology (matches EstraNet).

In [ ]:
# Evaluation configuration
checkpoint_idx = 100000  # Which checkpoint to evaluate (0 = latest)

eval_cmd = f"""python mamba-gnn-scripts/train_mamba_gnn.py \
    --data_path={config['data_path']} \
    --checkpoint_dir={config['checkpoint_dir']} \
    --target_byte={config['target_byte']} \
    --d_model={config['d_model']} \
    --mamba_layers={config['mamba_layers']} \
    --gnn_layers={config['gnn_layers']} \
    --k_neighbors={config['k_neighbors']} \
    --dropout={config['dropout']} \
    --checkpoint_idx={checkpoint_idx} \
    --result_path={RESULT_DIR}/mamba_gnn_eval
"""

print("Evaluation command:")
print(eval_cmd)
print("\n✓ This will compute 100-trial Guessing Entropy")
print("  (Takes ~10-15 minutes)")

Evaluation command:
python mamba-gnn-scripts/train_mamba_gnn.py     --data_path=data/ASCAD.h5     --checkpoint_dir=checkpoints/mamba_gnn_phase1_fair     --target_byte=2     --d_model=128     --mamba_layers=4     --gnn_layers=3     --k_neighbors=8     --dropout=0.1     --checkpoint_idx=100000     --result_path=results/mamba_gnn_eval


✓ This will compute 100-trial Guessing Entropy
  (Takes ~10-15 minutes)


In [ ]:
# Execute evaluation
import subprocess

print("="*80)
print("Evaluating Mamba-GNN (100-trial Guessing Entropy)")
print("="*80)

result = subprocess.run(eval_cmd.split(), capture_output=False)

if result.returncode == 0:
    print("\n✓ Evaluation complete!")
else:
    print("\n✗ Evaluation failed")

## Plot Guessing Entropy Curve

In [ ]:
# Load and plot evaluation results
result_file = f'{RESULT_DIR}/mamba_gnn_eval.txt'

if Path(result_file).exists():
    with open(result_file, 'r') as f:
        lines = f.readlines()
        mean_ranks = np.array([float(x) for x in lines[0].strip().split('\t')])
        std_ranks = np.array([float(x) for x in lines[1].strip().split('\t')])

    traces = np.arange(1, len(mean_ranks) + 1)

    # Create plot
    plt.figure(figsize=(14, 7))

    # Main line
    plt.plot(traces, mean_ranks, 'b-', linewidth=2.5, label='Mean Key Rank (GE)')

    # Confidence interval
    plt.fill_between(traces,
                     mean_ranks - std_ranks,
                     mean_ranks + std_ranks,
                     alpha=0.3, color='blue', label='±1 Std Dev')

    # Key recovered line
    plt.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Key Recovered (Rank=0)')

    # Labels and formatting
    plt.xlabel('Number of Traces', fontsize=14)
    plt.ylabel('Key Rank (Guessing Entropy)', fontsize=14)
    plt.title('Mamba-GNN: Guessing Entropy Evaluation (100 trials)\nEstraNet-Aligned Configuration',
              fontsize=16, fontweight='bold')
    plt.legend(fontsize=12, loc='upper right')
    plt.grid(True, alpha=0.3)

    # Add milestones
    milestones = [100, 500, 1000, 2000, 5000]
    for m in milestones:
        if m < len(mean_ranks):
            plt.axvline(x=m, color='gray', linestyle=':', alpha=0.5)
            plt.text(m, plt.ylim()[1] * 0.95, f'{m}',
                    ha='center', fontsize=9, color='gray')

    plt.tight_layout()
    plt.savefig(f'{RESULT_DIR}/guessing_entropy_curve.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Print summary statistics
    print("="*80)
    print("GUESSING ENTROPY RESULTS")
    print("="*80)
    print(f"\nTarget byte: {config['target_byte']}")
    print(f"\nKey Rank (Mean ± Std):")
    print(f"  100 traces:   {mean_ranks[99]:.2f} ± {std_ranks[99]:.2f}")
    print(f"  500 traces:   {mean_ranks[499]:.2f} ± {std_ranks[499]:.2f}")
    print(f"  1000 traces:  {mean_ranks[999]:.2f} ± {std_ranks[999]:.2f}")
    if len(mean_ranks) >= 2000:
        print(f"  2000 traces:  {mean_ranks[1999]:.2f} ± {std_ranks[1999]:.2f}")
    if len(mean_ranks) >= 5000:
        print(f"  5000 traces:  {mean_ranks[4999]:.2f} ± {std_ranks[4999]:.2f}")
    if len(mean_ranks) >= 10000:
        print(f"  10000 traces: {mean_ranks[9999]:.2f} ± {std_ranks[9999]:.2f}")

    # Find recovery point
    recovered_idx = np.where(mean_ranks == 0)[0]
    if len(recovered_idx) > 0:
        print(f"\n✓ Key RECOVERED at {recovered_idx[0]+1} traces")
    else:
        print(f"\n✗ Key NOT recovered (best rank: {mean_ranks[-1]:.2f})")

    print("\n" + "="*80)
    print(f"Plot saved: {RESULT_DIR}/guessing_entropy_curve.png")
    print("="*80)
else:
    print("⚠ No evaluation results found. Run evaluation first.")

## Compare with EstraNet

Compare Mamba-GNN with EstraNet Transformer/GNN models.

In [ ]:
# Compare with EstraNet results
estranet_result = 'results/estranet_transformer_eval.txt'  # Update this path

compare_cmd = f"""python scripts/compare_results.py \
    --mamba_results={result_file} \
    --estranet_results={estranet_result} \
    --output={RESULT_DIR}/model_comparison.png
"""

print("Comparison command:")
print(compare_cmd)
print("\n✓ Make sure you have EstraNet results first:")
print("  python scripts/train_trans.py --model_type=transformer --do_train")

In [ ]:
# Execute comparison (uncomment when EstraNet results are ready)
# import subprocess
# result = subprocess.run(compare_cmd.split())
# print("✓ Comparison plot created")

## Load and Use Trained Model

Load checkpoint for inference or further analysis.

In [ ]:
import torch
import h5py
from sklearn.preprocessing import StandardScaler

# Import model
sys.path.append('models')
from mamba_gnn_model import OptimizedMambaGNN

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model (EstraNet-aligned configuration)
model = OptimizedMambaGNN(
    trace_length=700,
    d_model=config['d_model'],
    mamba_layers=config['mamba_layers'],
    gnn_layers=config['gnn_layers'],
    num_classes=256,
    k_neighbors=config['k_neighbors'],
    dropout=config['dropout']
).to(device)

# Load checkpoint
ckpt_path = f"{CHECKPOINT_DIR}/mamba_gnn-100000.pth"

if Path(ckpt_path).exists():
    checkpoint = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    total_params = sum(p.numel() for p in model.parameters())

    print(f"\n✓ Model loaded successfully")
    print(f"  Checkpoint: {ckpt_path}")
    print(f"  Training step: {checkpoint['global_step']:,}")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Model ready for inference")
else:
    print(f"⚠ Checkpoint not found: {ckpt_path}")
    print("  Train the model first or specify different checkpoint_idx")

In [ ]:
import shutil
from google.colab import files

# Define paths
checkpoint_folder = 'checkpoints/mamba_gnn_estranet'
output_filename = 'mamba_gnn_estranet_checkpoints'

# Check if folder exists
if os.path.exists(checkpoint_folder):
    print(f"Zipping {checkpoint_folder}...")

    # Create zip file
    shutil.make_archive(output_filename, 'zip', checkpoint_folder)

    # Download
    zip_path = f"{output_filename}.zip"
    print(f"Downloading {zip_path} ({os.path.getsize(zip_path) / 1e6:.1f} MB)...")
    files.download(zip_path)
else:
    print(f"Folder not found: {checkpoint_folder}")

In [ ]:
# Example: Get predictions on attack traces
if Path(ckpt_path).exists() and Path(DATA_PATH).exists():
    print("Running inference on sample traces...")

    # Load attack data
    with h5py.File(DATA_PATH, 'r') as f:
        X_attack = f['Attack_traces/traces'][:100]  # First 100 traces
        m_attack = f['Attack_traces/metadata'][:]

    # Normalize (using same scaler as training)
    with h5py.File(DATA_PATH, 'r') as f:
        X_train_sample = f['Profiling_traces/traces'][:1000]

    scaler = StandardScaler()
    scaler.fit(X_train_sample)
    X_attack_norm = scaler.transform(X_attack)

    # Get predictions
    X_tensor = torch.FloatTensor(X_attack_norm).to(device)

    with torch.no_grad():
        logits = model(X_tensor)
        probs = torch.softmax(logits, dim=1)

    predicted_classes = torch.argmax(probs, dim=1).cpu().numpy()

    print(f"\n✓ Inference complete")
    print(f"  Traces processed: {len(X_attack)}")
    print(f"  Prediction shape: {probs.shape}")
    print(f"  Top predicted classes: {predicted_classes[:10]}")
    print(f"  Confidence (first trace): {probs[0].max().item():.4f}")
else:
    print("⚠ Model or data not available")

## Summary & Configuration Verification

**✅ This notebook uses the CORRECT configuration:**

In [ ]:
# Verify configuration matches EstraNet
print("="*80)
print("CONFIGURATION VERIFICATION")
print("="*80)

verification = {
    'Loss Function': ('Cross-Entropy', '✓'),
    'Learning Rate': ('2.5e-4', '✓'),
    'Train Batch Size': ('256', '✓'),
    'Eval Batch Size': ('32', '✓'),
    'Training Steps': ('100,000', '✓'),
    'Optimizer': ('Adam', '✓'),
    'LR Schedule': ('Cosine Decay', '✓'),
    'Model Dimension': ('128', '✓'),
    'Dropout': ('0.1', '✓'),
    'Gradient Clipping': ('0.25', '✓'),
    'Evaluation Method': ('100-trial GE', '✓'),
}

print("\nEstraNet Alignment Check:")
for param, (value, status) in verification.items():
    print(f"  {status} {param:25s}: {value}")

print("\n" + "="*80)
print("✓ ALL PARAMETERS MATCH ESTRANET")
print("✓ FAIR COMPARISON GUARANTEED")
print("="*80)

print("\n⚠️ DO NOT USE: final_best_gnn_mamba_teacher.ipynb")
print("   That notebook has incompatible configuration:")
print("   - FocalLoss instead of Cross-Entropy")
print("   - Learning rate 8x too high")
print("   - Batch size 4x too small")
print("   - Single-trial evaluation instead of 100-trial GE")

print("\n✓ USE THIS NOTEBOOK for training and evaluation")

## Next Steps

### 1. Train Model
Run the training cell above to start training with correct configuration.

### 2. Monitor Progress
Check training loss, learning rate, and gradient norms periodically.

### 3. Evaluate Model
After training, run evaluation to get Guessing Entropy curves.

### 4. Compare with EstraNet
Train EstraNet models and compare results using the comparison script.

### 5. Expected Results
- **After 100k steps**: Loss ~4.8-5.0
- **Key recovery**: Within 1000-2000 traces
- **Performance**: Comparable to EstraNet Transformer (~1200 traces)

### Files Created
- **Training script**: `mamba-gnn-scripts/train_mamba_gnn.py`
- **TensorFlow script**: `scripts/train_mamba_gnn_tf.py`
- **PowerShell runner**: `mamba-gnn-scripts/train_mamba_gnn.ps1`
- **Bash runner**: `mamba-gnn-scripts/train_mamba_gnn.sh`
- **Comparison tool**: `scripts/compare_results.py`
- **This notebook**: `train_mamba_gnn_notebook.ipynb`

### Documentation
- **Quick start**: `QUICKSTART.md`
- **TFLite guide**: `TENSORFLOW_TFLITE_GUIDE.md`
- **TF implementation**: `TENSORFLOW_IMPLEMENTATION_SUMMARY.md`
- **Config comparison**: `NOTEBOOK_VS_SCRIPT_COMPARISON.md`

**Ready to train with fair comparison to EstraNet!** 🎯

## Compare Both Phases for Report

After completing Phase 1️⃣ and Phase 2️⃣, use this to compare results.


In [ ]:
import pandas as pd

# Load results from both phases
phase1_result = f'{RESULT_DIR}/cross_entropy_eval.txt'
phase2_result = f'{RESULT_DIR}/focal_loss_eval.txt'

results = {}

# Phase 1: Fair Comparison
if Path(phase1_result).exists():
    with open(phase1_result, 'r') as f:
        lines = f.readlines()
        phase1_mean = np.array([float(x) for x in lines[0].strip().split('\t')])
        phase1_std = np.array([float(x) for x in lines[1].strip().split('\t')])
    results['Phase 1 (Cross-Entropy)'] = phase1_mean
    print("✓ Phase 1 results loaded")
else:
    print("⚠ Phase 1 results not found. Train Phase 1 first.")

# Phase 2: Performance
if Path(phase2_result).exists():
    with open(phase2_result, 'r') as f:
        lines = f.readlines()
        phase2_mean = np.array([float(x) for x in lines[0].strip().split('\t')])
        phase2_std = np.array([float(x) for x in lines[1].strip().split('\t')])
    results['Phase 2 (FocalLoss)'] = phase2_mean
    print("✓ Phase 2 results loaded")
else:
    print("⚠ Phase 2 results not found. Train Phase 2 first.")

# Compare key milestones
if len(results) == 2:
    print("\n" + "="*80)
    print("COMPARISON TABLE FOR REPORT")
    print("="*80)

    milestones = [100, 500, 1000, 2000, 5000]

    print(f"\n{'Traces':<10} {'Phase 1 (CE)':<20} {'Phase 2 (Focal)':<20} {'Improvement'}")
    print("-" * 70)

    for m in milestones:
        if m <= len(phase1_mean):
            p1_rank = phase1_mean[m-1]
            p2_rank = phase2_mean[m-1]
            improvement = ((p1_rank - p2_rank) / (p1_rank + 1e-6)) * 100

            print(f"{m:<10} {p1_rank:<20.2f} {p2_rank:<20.2f} {improvement:+.1f}%")

    print("\n" + "="*80)

    # Find recovery points
    p1_recovery = np.where(phase1_mean == 0)[0]
    p2_recovery = np.where(phase2_mean == 0)[0]

    print("\nKEY RECOVERY SUMMARY:")
    if len(p1_recovery) > 0:
        print(f"  Phase 1 (Cross-Entropy): {p1_recovery[0]+1} traces")
    else:
        print(f"  Phase 1 (Cross-Entropy): No recovery (best: {phase1_mean[-1]:.2f})")

    if len(p2_recovery) > 0:
        print(f"  Phase 2 (FocalLoss):     {p2_recovery[0]+1} traces")
    else:
        print(f"  Phase 2 (FocalLoss):     No recovery (best: {phase2_mean[-1]:.2f})")

    print("\n" + "="*80)

    # Plot comparison
    traces = np.arange(1, min(len(phase1_mean), len(phase2_mean)) + 1)

    plt.figure(figsize=(14, 7))
    plt.plot(traces, phase1_mean[:len(traces)], 'b-', linewidth=2.5, label='Phase 1: Cross-Entropy (Fair)')
    plt.plot(traces, phase2_mean[:len(traces)], 'r-', linewidth=2.5, label='Phase 2: FocalLoss (Optimized)')

    plt.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    plt.xlabel('Number of Traces', fontsize=14)
    plt.ylabel('Key Rank (Guessing Entropy)', fontsize=14)
    plt.title('Mamba-GNN Comparison: Fair vs Optimized\nfor Report', fontsize=16, fontweight='bold')
    plt.legend(fontsize=12, loc='upper right')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{RESULT_DIR}/phase_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"Comparison plot saved: {RESULT_DIR}/phase_comparison.png")
else:
    print("⚠ Complete both Phase 1 and Phase 2 to see comparison")